# ml_prototype_at_scale
1. In this notebook, It mainly focus on scaling the ML prototype to fit on large dataset.
2. I plan to do basic preprocessing, and then take text embeddings to vectorize the text document.
3. Next, I can utilize Spark ML and TensorFlow to create classifier, respectively

# 1 import basic library

In [3]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType
import pyspark.sql.functions as F

import matplotlib.pyplot as plt

# 2 get familary with spark session, load dataset and create catalog

## 2.1 spark session and version check

In [6]:
if 'spark' not in locals():
  print('[INFo] prepare environment and import library....')
  from pyspark.context import SparkContext
  from pyspark.sql.context import SQLContext
  from pyspark.sql.session import SparkSession
  
  spark = SparkSession()
else:
  print('[INFO] spark session is ready to use!')
  print(spark)

[INFO] spark session is ready to use!
<pyspark.sql.session.SparkSession object at 0x7ff3b8a737b8>

In [7]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [8]:
print('list all tables:', spark.sql('show tables').toPandas())
print('\n')
print('list all databases:', spark.sql('show databases').toPandas())

list all tables: Empty DataFrame
Columns: [database, tableName, isTemporary]
Index: []


list all databases: databaseName
0 default

In [9]:
%fs ls dbfs:/FileStore/shared_uploads/pipi41@163.com/

path,name,size
dbfs:/FileStore/shared_uploads/pipi41@163.com/multidomain_raw.csv,multidomain_raw.csv,31546876
dbfs:/FileStore/shared_uploads/pipi41@163.com/phasbank_raw.csv,phasbank_raw.csv,652545


## 2.2 load Multidomai dataset

In [11]:
pb_csv_file_path = 'dbfs:/FileStore/shared_uploads/pipi41@163.com/phasbank_raw.csv'
md_csv_file_path = 'dbfs:/FileStore/shared_uploads/pipi41@163.com/multidomain_raw.csv'

In [12]:
# define MultiDomain dataset structure and load the dataset csv into spark dataframe

schema_md = StructType([
  StructField('text', StringType()),
  StructField('target', IntegerType()),
  StructField('pre_clean_len', IntegerType())
])

md_raw_df = spark.read.format('csv').load(md_csv_file_path, schema=schema_md, sep="|", header=True)

In [13]:
print('Total line of MD_RAW is:', md_raw_df.count())
md_raw_df.show(5)

Total line of MD_RAW is: 38548
+--------------------+------+-------------+
 text|target|pre_clean_len|
+--------------------+------+-------------+
"I've always held...| 0| 3933|
someone get this ...| 0| 565|
Tihs Album is not...| 0| 218|
"this industry is...| 0| 87|
I'm sorry but the...| 0| 269|
+--------------------+------+-------------+
only showing top 5 rows

## 2.3 load phaseBank dataset

In [15]:
# define MultiDomain dataset structure and load the dataset csv into spark dataframe

schema_pb = StructType([
  StructField('text', StringType()),
  StructField('target', IntegerType()),
  StructField('pre_clean_len', IntegerType())
])

pb_raw_df = spark.read.format('csv').load(pb_csv_file_path, schema=schema_pb, sep="|", header=True)

In [16]:
print('[INFO] import phaseBank dataset. Total line of PB_RAW is:', pb_raw_df.count())
pb_raw_df.show(5)

[INFO] import phaseBank dataset. Total line of PB_RAW is: 4994
+--------------------+------+-------------+
 text|target|pre_clean_len|
+--------------------+------+-------------+
According to Gran...| -1| 127|
For the last quar...| 1| 193|
In the third quar...| 1| 125|
Operating profit ...| 1| 122|
Operating profit ...| 1| 103|
+--------------------+------+-------------+
only showing top 5 rows

In [17]:
pb_raw_df.printSchema(), md_raw_df.printSchema()

root
-- text: string (nullable = true)
-- target: integer (nullable = true)
-- pre_clean_len: integer (nullable = true)

root
-- text: string (nullable = true)
-- target: integer (nullable = true)
-- pre_clean_len: integer (nullable = true)

Out[11]: (None, None)

In [18]:
pb_raw_df.filter(pb_raw_df.pre_clean_len<15).show()

+--------------+------+-------------+
 text|target|pre_clean_len|
+--------------+------+-------------+
 Welcome !| -1| 9|
 It 's not .| -1| 11|
NWC ANALYSIS :| -1| 14|
 R&D Loan ) .| -1| 12|
 Welcome !| -1| 9|
 It 's not .| -1| 11|
NWC ANALYSIS :| -1| 14|
 R&D Loan ) .| -1| 12|
+--------------+------+-------------+

## 2.4 create temparary table for SQL query
1. Using "pb_df" as the table's name for Phrasebank dataset 
2. Using "md_df" as the table's name for Multidomain dataset

In [20]:
def run_sql(statement):
  try:
    res = spark.sql(statement)
  except Exception as e:
    print(e.desc, '\n', e.stackTrace)
    return 
  return res

In [21]:
pb_raw_df.createOrReplaceTempView("pb_df")
md_raw_df.createOrReplaceTempView("md_df")

In [22]:
print(run_sql('desc table pb_df').show(), run_sql('desc table md_df').show())

+-------------+---------+-------+
 col_name|data_type|comment|
+-------------+---------+-------+
 text| string| null|
 target| int| null|
pre_clean_len| int| null|
+-------------+---------+-------+

+-------------+---------+-------+
 col_name|data_type|comment|
+-------------+---------+-------+
 text| string| null|
 target| int| null|
pre_clean_len| int| null|
+-------------+---------+-------+

None None

# 3 DataFrame operation and SQL query operation
1. practice the basic operation using DataFrame and SQL
2. select, filter, groupBy, agg...

In [24]:
# select pre_clean_len, and sort descending 
print("DataFrame SELECT oper:", pb_raw_df.select(pb_raw_df.pre_clean_len)
                                         .sort('pre_clean_len', ascending=False)
                                         .show(5))
print(run_sql(
  '''
  SELECT pre_clean_len
    FROM pb_df
   ORDER BY pre_clean_len DESC
  '''
).show(5))

+-------------+
pre_clean_len|
+-------------+
 315|
 315|
 306|
 306|
 302|
+-------------+
only showing top 5 rows

DataFrame SELECT oper: None
+-------------+
pre_clean_len|
+-------------+
 315|
 315|
 306|
 306|
 302|
+-------------+
only showing top 5 rows

None

In [25]:
# filter 
print(pb_raw_df.filter(pb_raw_df.pre_clean_len<50).show(5))
print(run_sql('''
  SELECT *
    FROM pb_df
   WHERE pre_clean_len<50
''').show(5))

+--------------------+------+-------------+
 text|target|pre_clean_len|
+--------------------+------+-------------+
Sales have risen ...| 1| 42|
Cargo volume incr...| 1| 45|
Diluted EPS rose ...| 1| 44|
EBIT margin was u...| 1| 40|
EPS grew to 0.04 ...| 1| 36|
+--------------------+------+-------------+
only showing top 5 rows

None
+--------------------+------+-------------+
 text|target|pre_clean_len|
+--------------------+------+-------------+
Sales have risen ...| 1| 42|
Cargo volume incr...| 1| 45|
Diluted EPS rose ...| 1| 44|
EBIT margin was u...| 1| 40|
EPS grew to 0.04 ...| 1| 36|
+--------------------+------+-------------+
only showing top 5 rows

None

In [26]:
print(pb_raw_df.groupBy().min('pre_clean_len').show())
print(pb_raw_df.groupBy().max('pre_clean_len').show())
print(pb_raw_df.groupBy().avg('pre_clean_len').show())

+------------------+
min(pre_clean_len)|
+------------------+
 9|
+------------------+

None
+------------------+
max(pre_clean_len)|
+------------------+
 315|
+------------------+

None
+------------------+
avg(pre_clean_len)|
+------------------+
123.32759311173407|
+------------------+

None

In [27]:
display(pb_raw_df.select('pre_clean_len').toPandas())

pre_clean_len
127
193
125
122
103
163
143
163
254
237


# 4 data preprocessing and embedding

## 4.1 import spark-nlp library

In [30]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *

In [31]:
from pyspark.ml.classification import LogisticRegression

## 4.2 split dataset

In [33]:
pb_raw_df.groupBy("target").count().show()

+------+-----+
target|count|
+------+-----+
 1| 1457|
 0| 3537|
+------+-----+

In [34]:
from pyspark.sql.functions import when

pb_raw_df = pb_raw_df.withColumn("target", when(pb_raw_df.target==-1, 0).otherwise(1))

In [35]:
(train_pb, test_pb) = pb_raw_df.randomSplit([0.75, 0.25], seed=42)
(train_md, test_md) = md_raw_df.randomSplit([0.75, 0.25], seed=42)

In [36]:
print('phasebank dataset positive percentage is {}'.format(pb_raw_df.filter(pb_raw_df.target==1).count() / pb_raw_df.count()))
print('multidomain dataset positive percentage is {}'.format(md_raw_df.filter(md_raw_df.target==1).count()/md_raw_df.count()))

print('phasebank train dataset positive percentage is {}'.format(train_pb.filter(train_pb.target==1).count()/train_pb.count()))
print('phasebank test dataset positive percentage is {}'.format(test_pb.filter(test_pb.target==1).count() / test_pb.count()))
print('multidomain train dataset positive percentage is {}'.format(train_md.filter(train_md.target==1).count()/train_md.count()))
print('multidomain test dataset positive percentage is {}'.format(test_md.filter(test_md.target==1).count() / test_md.count()))

phasebank dataset positive percentage is 0.2917501001201442
multidomain dataset positive percentage is 0.5699906609940852
phasebank train dataset positive percentage is 0.28582760444209415
phasebank test dataset positive percentage is 0.3102310231023102
multidomain train dataset positive percentage is 0.5703782962860693
multidomain test dataset positive percentage is 0.5688178780284043

## 4.3 define the spark nlp pipeline

In [38]:
# prepare pipeline stage for future construction


# main entry point pipeline for most for spark-nlp functionality
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")


# detect sentences in document columns
sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")


# tokenize the input with a standard tokenizer
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")


# normalize the tokens
normalizer = Normalizer().setInputCols(['token']).setOutputCol('normalized').setCleanupPatterns(["[^\w\d\s]"])


# # token assembler, getting data reshape
# token_assembler = TokenAssembler().setInputCols(['document', 'normalized']).setOutputCol('clean_text')


# stopword cleaning
# stopwords_cleaner = StopWordsCleaner().setInputCols("normalized").setOutputCol("cleanedToken").setCaseSensitive(False)


# define elmo model from spark-nlp repo
word_embedding_elmo = ElmoEmbeddings.pretrained().setInputCols(['normalized', 'document']).setOutputCol('elmo')


# define BERT embedding
word_embedding_bert = BertEmbeddings.pretrained('bert_base_cased', 'en').setInputCols(['normalized', 'document']).setOutputCol('bert')


# senetnce embedding, use emlo and bert 
sentence_embedding_elmo = SentenceEmbeddings().setInputCols(['document', 'elmo']).setOutputCol('sentence_embeddings').setPoolingStrategy('AVERAGE')
sentence_embedding_bert = SentenceEmbeddings().setInputCols(['document', 'bert']).setOutputCol('sentence_embeddings').setPoolingStrategy('AVERAGE')


# define embeddings finisher, no matter which embedding you selected, elmo or bert, both from the same column_name
embeddings_finisher = EmbeddingsFinisher().setInputCols(["sentence_embeddings"]).setOutputCols(["finished_sentence_embeddings"]).setCleanAnnotations(False).setOutputAsVector(True)

# define classifier
classifierdl = ClassifierDLApproach().setInputCols(['sentence_embeddings']).setOutputCol('class').setLabelColumn('target').setMaxEpochs(5).setEnableOutputLogs(True)


# elmo-embedding pipeline
embedding_emlo_pipeline = Pipeline(stages=[document_assembler,
                                           sentence_detector,
                                           tokenizer,
                                           normalizer,
                                           word_embedding_elmo,
                                           sentence_embedding_elmo,
                                           embeddings_finisher])

# bert-embedding pipeline
embedding_bert_pipeline = Pipeline(stages=[document_assembler,
                                           sentence_detector,
                                           tokenizer,
                                           normalizer,
                                           word_embedding_bert,
                                           sentence_embedding_bert,
                                           embeddings_finisher])

elmo download started this may take some time.
Approximate size to download 334.1 MB
[ | ][OK!]
bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[ | ][OK!]

## 4.4 pipeline exploration
* check the step result in pipeline to get familiar with the whole process.

In [40]:
# define empty dataframe
empty_df = spark.createDataFrame([['']]).toDF("text")
text = [['( ADP News ) - Feb 11 , 2009 - Finnish management is a software solutions provider. Ixonos Oyj ( HEL : XNS1V ) said today its net profit rose to EUR 3.5 million ( USD 4.5 m ) for 2008 from EUR 3.1 million for 2007 .'], 
        ['( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . )']]
test_df = spark.createDataFrame(text, StringType()).toDF("text")

In [41]:
# detect the sentence
sentence_pipeline = Pipeline(stages=[document_assembler,
                                     sentence_detector])
sentence_pipelineModel = sentence_pipeline.fit(empty_df)

result = sentence_pipelineModel.transform(train_pb.limit(10))

# result_df = result.select(F.explode(F.arrays_zip('token.result', 'elmo.embeddings')).alias('cols')).select(F.expr("cols['0']").alias("token"), F.expr("cols['1']").alias("elmo_embedding"))
display(result)

text,target,pre_clean_len,document,sentence
( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ),-1,78,"List(List(document, 0, 77, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ), Map(sentence -> 0), List()))","List(List(document, 0, 75, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor ., Map(sentence -> 0), List()), List(document, 77, 77, ), Map(sentence -> 1), List()))"
"( ADP News ) - Feb 11 , 2009 - Finnish management software solutions provider Ixonos Oyj ( HEL : XNS1V ) said today its net profit rose to EUR 3.5 million ( USD 4.5 m ) for 2008 from EUR 3.1 million for 2007 .",1,209,"List(List(document, 0, 208, ( ADP News ) - Feb 11 , 2009 - Finnish management software solutions provider Ixonos Oyj ( HEL : XNS1V ) said today its net profit rose to EUR 3.5 million ( USD 4.5 m ) for 2008 from EUR 3.1 million for 2007 ., Map(sentence -> 0), List()))","List(List(document, 0, 208, ( ADP News ) - Feb 11 , 2009 - Finnish management software solutions provider Ixonos Oyj ( HEL : XNS1V ) said today its net profit rose to EUR 3.5 million ( USD 4.5 m ) for 2008 from EUR 3.1 million for 2007 ., Map(sentence -> 0), List()))"
"( ADP News ) - Feb 11 , 2009 - Finnish management software solutions provider Ixonos Oyj ( HEL : XNS1V ) said today its net profit rose to EUR 3.5 million ( USD 4.5 m ) for 2008 from EUR 3.1 million for 2007 .",1,209,"List(List(document, 0, 208, ( ADP News ) - Feb 11 , 2009 - Finnish management software solutions provider Ixonos Oyj ( HEL : XNS1V ) said today its net profit rose to EUR 3.5 million ( USD 4.5 m ) for 2008 from EUR 3.1 million for 2007 ., Map(sentence -> 0), List()))","List(List(document, 0, 208, ( ADP News ) - Feb 11 , 2009 - Finnish management software solutions provider Ixonos Oyj ( HEL : XNS1V ) said today its net profit rose to EUR 3.5 million ( USD 4.5 m ) for 2008 from EUR 3.1 million for 2007 ., Map(sentence -> 0), List()))"
"( ADP News ) - Feb 12 , 2009 - Finnish IT solutions provider Affecto Oyj ( HEL : AFE1V ) said today its net profit rose to EUR 8.5 million ( USD 11m ) in 2008 from EUR 7 million in 2007 .",1,187,"List(List(document, 0, 186, ( ADP News ) - Feb 12 , 2009 - Finnish IT solutions provider Affecto Oyj ( HEL : AFE1V ) said today its net profit rose to EUR 8.5 million ( USD 11m ) in 2008 from EUR 7 million in 2007 ., Map(sentence -> 0), List()))","List(List(document, 0, 186, ( ADP News ) - Feb 12 , 2009 - Finnish IT solutions provider Affecto Oyj ( HEL : AFE1V ) said today its net profit rose to EUR 8.5 million ( USD 11m ) in 2008 from EUR 7 million in 2007 ., Map(sentence -> 0), List()))"
"( ADP News ) - Feb 12 , 2009 - Finnish IT solutions provider Affecto Oyj ( HEL : AFE1V ) said today its net profit rose to EUR 8.5 million ( USD 11m ) in 2008 from EUR 7 million in 2007 .",1,187,"List(List(document, 0, 186, ( ADP News ) - Feb 12 , 2009 - Finnish IT solutions provider Affecto Oyj ( HEL : AFE1V ) said today its net profit rose to EUR 8.5 million ( USD 11m ) in 2008 from EUR 7 million in 2007 ., Map(sentence -> 0), List()))","List(List(document, 0, 186, ( ADP News ) - Feb 12 , 2009 - Finnish IT solutions provider Affecto Oyj ( HEL : AFE1V ) said today its net profit rose to EUR 8.5 million ( USD 11m ) in 2008 from EUR 7 million in 2007 ., Map(sentence -> 0), List()))"
"( ADP News ) - Feb 6 , 2009 - Finnish fishing tackle company Rapala VMC Corp ( HEL : RAP1V ) said today its net profit rose to EUR 19.2 million ( USD 24.6 m ) for 2008 from EUR 17.5 million for 2007 .",1,200,"List(List(document, 0, 199, ( ADP News ) - Feb 6 , 2009 - Finnish fishing tackle company Rapala VMC Corp ( HEL : RAP1V ) said today its net profit rose to EUR 19.2 million ( USD 24.6 m ) for 2008 from EUR 17.5 million for 2007 ., Map(sentence -> 0), List()))","List(List(document, 0, 199, ( ADP News ) - Feb 6 , 2009 - Finnish fishing tackle company Rapala VMC Corp ( HEL : RAP1V ) said today its net profit rose to EUR 19.2 million

In [42]:
# try normalized the data 
normalized_pipeline = Pipeline(stages=[document_assembler,
                                       sentence_detector,
                                       tokenizer,
                                       normalizer])
normalizer_pipelineModel = normalized_pipeline.fit(empty_df)
result = normalizer_pipelineModel.transform(train_pb.limit(2))
# display(result)
display(result.withColumn("tmp", F.explode(F.arrays_zip("normalized.result", "document"))).select("tmp.*")) # select('begin', 'end', 'result', 'metadata.sentence').show()

0,document
A,"List(document, 0, 77, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ), Map(sentence -> 0), List())"
spokesperson,null
told,null
WWD,null
to,null
expect,null
a,null
5050,null
mix,null
of,null


In [43]:
# # try tokenAssembler
# tokenassembled_pipeline = Pipeline(stages=[document_assembler,
#                                            sentence_detector,
#                                            tokenizer,
#                                            normalizer,
#                                            token_assembler])
# tokenassembled_pipelineModel = tokenassembled_pipeline.fit(empty_df)
# result = tokenassembled_pipelineModel.transform(test_df)
# tmp = result.withColumn("tmp", F.explode("clean_text")).select("tmp.*").select("metadata.sentence", "result")
# # display(tmp)
# tmp = result.withColumn("tmp", F.explode("clean_text")).select("tmp.*").show()

In [44]:
# word embedding and sentence embedding with elmo
embedding_emlo_pipelineModel = embedding_emlo_pipeline.fit(empty_df)
result = embedding_emlo_pipelineModel.transform(train_pb.limit(2))
word_embedding = result.withColumn("tmp", F.explode("elmo")).select("tmp.*")
sentence_embedding = result.withColumn("tmp", F.explode("sentence_embeddings")).select("tmp.*")
# embeddings_finisher = result.withColumn("tmp", F.explode())
display(result)

text target pre_clean_len document sentence token normalized elmo sentence_embeddings finished_sentence_embeddings ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ) -1 78 List(List(document, 0, 77, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ), Map(sentence -> 0), List())) List(List(document, 0, 75, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor ., Map(sentence -> 0), List()), List(document, 77, 77, ), Map(sentence -> 1), List())) List(List(token, 0, 0, (, Map(sentence -> 0), List()), List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 42, 50-50, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List()), List(token, 75, 75, ., Map(sentence -> 0), List()), List(token, 77, 77, ), Map(sentence -> 0), List())) List(List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 41, 5050, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List())) List(List(word_embeddings, 2, 2, A, Map(isOOV -> false, pieceId -> -1, isWordStart -> true, token -> A, sentence -> 0), List(0.16875874, -0.09871332, -1.0668647, -0.3367843, 0.056462966, 0.072602026, -0.24000625, 0.28351867, 0.2292295, -0.2329422, 0.0894687, -0.40849942, -0.10388976, 0.4859494, -0.18489513, -0.28658837, 0.2854353, 0.25814894, -0.10831635, 0.05919902, 0.02652353, -0.13388264, 0.40880495, 0.6657118, -0.17387958, -0.14374793, -0.93987715, 0.49303353, 0.116646, 0.34871072, -0.4564208, -0.34849358, -0.05685176, -0.14319275, -0.052185576, -0.39066386, -0.36120912, 0.45699096, -0.2880788, -0.7195122, 0.25580114, -0.3249084, 0.16236782, 0.31407604, 0.011028601, 0.61614496, -0.489135, 0.07032255, -0.25694162, -0.16355476, -0.15162623, 0.017138865, 0.18795368, 0.39603338, 0.1683697, 0.47688764, -0.09720263, 0.71960264, -0.4419365, -0.13757558, 0.29675266, 0.07524943, 0.38240185, 0.09795726, -0.16672157, -0.11879185, -0.033043236, 0.46737024, -0.4656591, 0.013470706, -0.3052773, 0.2774338, 0.9665144, 0.2133083, 0.15156062, -0.110642806, 0.11173977, 1.0575278, -0.3842764, 0.07276094, 0.13288063, 0.35702062, -0.17947792, -0.8514069, -0.21485177, -0.03285347, -0.114653915, -0.16667715, 0.26668265, 0.10868572, 0.20945226, -0.20951378, -0.5638415, 0.47642305, 0.3639202, -0.2660443, -0.38415313, -0.396048, 0.2944262, 0.43490162, -0.23271376, -0.29697222, 0.19780251, -0.08239995, 2.0316243E-4, -0.4736708, 0.34313375, 0.70314604, -0.031462878, -0.3893208, 0.31646442, -0.6528768, -0.355967, 0.30528912, -0.26065147, -0.10254332, -0.13084313, -0.08311957, 0.08426628, -0.1550655, -0.18172368, -0.09685904, 0.36711097, 0.29362333, 0.065880686, -0.14145055, -0.26337516, 0.53699714, -0.0513601, 0.4338913, -0.528458, 0.12269708, 0.015314408, -0.42515165

In [45]:
# word embedding and sentence embedding with bert
embedding_bert_pipelineModel = embedding_bert_pipeline.fit(empty_df)
result = embedding_bert_pipelineModel.transform(train_pb.limit(2))
display(result)

text target pre_clean_len document sentence token normalized bert sentence_embeddings finished_sentence_embeddings ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ) -1 78 List(List(document, 0, 77, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ), Map(sentence -> 0), List())) List(List(document, 0, 75, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor ., Map(sentence -> 0), List()), List(document, 77, 77, ), Map(sentence -> 1), List())) List(List(token, 0, 0, (, Map(sentence -> 0), List()), List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 42, 50-50, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List()), List(token, 75, 75, ., Map(sentence -> 0), List()), List(token, 77, 77, ), Map(sentence -> 0), List())) List(List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 41, 5050, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List())) List(List(word_embeddings, 2, 2, A, Map(isOOV -> false, pieceId -> 138, isWordStart -> true, token -> A, sentence -> 0), List(-0.0591245, 0.20256183, 0.8251248, 0.78046286, -1.7807933, -0.80155766, -1.1157473, -0.4782777, -0.11486364, -0.5550404, -0.4492663, -0.62475693, -0.9387575, -0.98547107, -0.0060980394, -1.7143714, 0.9728724, 0.8024508, -0.11146159, 0.27530736, -0.7498719, 0.3653813, -1.1369629, 0.7400096, 0.87870675, 0.67104024, -0.6720965, -0.97241056, -0.087701544, 0.11403816, -1.3198427, -2.1072302, -0.45286688, -0.95254016, 0.2114704, 0.32399213, -0.95956004, -0.67034465, -1.0145385, -0.1401028, 0.6138847, -0.008340925, -0.5912232, -0.042774387, -0.41416043, -0.305131, -0.5699813, 0.683128, 0.4442352, -1.2327226, -0.8284215, -0.6327381, -0.8909469, -0.34239066, -0.4762854, -1.5815666, 0.23999412, 0.6981139, -0.024432093, 0.14011878, -0.65106165, 0.7304999, 0.6718264, -0.27875805, 0.2163485, 0.33989796, -1.0445671, -0.06548089, 0.010210072, 0.247049, 0.6969329, -0.4022462, 0.77358055, -0.57319415, 0.42015353, 1.1350511, -0.16513473, -0.61211205, -0.4262871, -0.4906991, 0.73714745, -0.091579914, 1.0832553, 0.43265566, -0.9382083, 0.092016555, 0.12843782, -0.6982337, 0.17024484, 0.45182657, 0.30771467, 0.55378497, 0.34565946, -0.5830273, 1.3283966, 0.12674963, -0.23364174, 0.29585448, -0.5420519, 0.34525484, -0.5825339, -0.35446483, -0.2273812, -0.5216397, -1.2295258, 1.2310001, -0.43397206, -0.47614375, -0.8801553, -0.5638279, 0.63400173, -0.2587341, 0.10292122, -1.7655771, -0.2949517, 0.9170294, 0.54532516, 0.45913243, 0.32567474, -0.14687663, -0.5588464, -0.24423473, -1.1699482, 0.6929512, 0.66577256, 0.24080604, 0.41866362, 0.7635331, -0.3947026, 0.30512983, 0.8840208, 0.3868008, 0.6842996, 0.5227849, 0.22083575, 0.95339185, -1.3

## 4.5 generate embedding
* embedding_emlo_pipeline 
* embedding_bert_pipeline

In [47]:
#transform train set
train_elmo_embedding_df = embedding_emlo_pipeline.fit(train_pb).transform(train_pb)
train_elmo_embedding_df = train_elmo_embedding_df.withColumn("feature_elmo", F.explode(train_elmo_embedding_df.finished_sentence_embeddings))

# transform test set
test_elmo_embedding_df = embedding_emlo_pipeline.fit(test_pb).transform(test_pb)
test_elmo_embedding_df = test_elmo_embedding_df.withColumn("feature_elmo", F.explode(test_elmo_embedding_df.finished_sentence_embeddings))

In [48]:
display(train_elmo_embedding_df.limit(5))

text target pre_clean_len document sentence token normalized elmo sentence_embeddings finished_sentence_embeddings feature_elmo ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ) 0 78 List(List(document, 0, 77, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ), Map(sentence -> 0), List())) List(List(document, 0, 75, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor ., Map(sentence -> 0), List()), List(document, 77, 77, ), Map(sentence -> 1), List())) List(List(token, 0, 0, (, Map(sentence -> 0), List()), List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 42, 50-50, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List()), List(token, 75, 75, ., Map(sentence -> 0), List()), List(token, 77, 77, ), Map(sentence -> 0), List())) List(List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 41, 5050, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List())) List(List(word_embeddings, 2, 2, A, Map(isOOV -> false, pieceId -> -1, isWordStart -> true, token -> A, sentence -> 0), List(0.16875874, -0.09871332, -1.0668647, -0.3367843, 0.056462966, 0.072602026, -0.24000625, 0.28351867, 0.2292295, -0.2329422, 0.0894687, -0.40849942, -0.10388976, 0.4859494, -0.18489513, -0.28658837, 0.2854353, 0.25814894, -0.10831635, 0.05919902, 0.02652353, -0.13388264, 0.40880495, 0.6657118, -0.17387958, -0.14374793, -0.93987715, 0.49303353, 0.116646, 0.34871072, -0.4564208, -0.34849358, -0.05685176, -0.14319275, -0.052185576, -0.39066386, -0.36120912, 0.45699096, -0.2880788, -0.7195122, 0.25580114, -0.3249084, 0.16236782, 0.31407604, 0.011028601, 0.61614496, -0.489135, 0.07032255, -0.25694162, -0.16355476, -0.15162623, 0.017138865, 0.18795368, 0.39603338, 0.1683697, 0.47688764, -0.09720263, 0.71960264, -0.4419365, -0.13757558, 0.29675266, 0.07524943, 0.38240185, 0.09795726, -0.16672157, -0.11879185, -0.033043236, 0.46737024, -0.4656591, 0.013470706, -0.3052773, 0.2774338, 0.9665144, 0.2133083, 0.15156062, -0.110642806, 0.11173977, 1.0575278, -0.3842764, 0.07276094, 0.13288063, 0.35702062, -0.17947792, -0.8514069, -0.21485177, -0.03285347, -0.114653915, -0.16667715, 0.26668265, 0.10868572, 0.20945226, -0.20951378, -0.5638415, 0.47642305, 0.3639202, -0.2660443, -0.38415313, -0.396048, 0.2944262, 0.43490162, -0.23271376, -0.29697222, 0.19780251, -0.08239995, 2.0316243E-4, -0.4736708, 0.34313375, 0.70314604, -0.031462878, -0.3893208, 0.31646442, -0.6528768, -0.355967, 0.30528912, -0.26065147, -0.10254332, -0.13084313, -0.08311957, 0.08426628, -0.1550655, -0.18172368, -0.09685904, 0.36711097, 0.29362333, 0.065880686, -0.14145055, -0.26337516, 0.53699714, -0.0513601, 0.4338913, -0.528458, 0.12269708, 0.015314408,

In [49]:
train_bert_embedding_df = embedding_bert_pipeline.fit(train_pb).transform(train_pb)
train_bert_embedding_df = train_bert_embedding_df.withColumn("feature_bert", F.explode(train_bert_embedding_df.finished_sentence_embeddings))

test_bert_embedding_df = embedding_bert_pipeline.fit(test_pb).transform(test_pb)
test_bert_embedding_df = test_bert_embedding_df.withColumn("feature_bert", F.explode(test_bert_embedding_df.finished_sentence_embeddings))

In [50]:
display(train_bert_embedding_df.limit(5))

text target pre_clean_len document sentence token normalized bert sentence_embeddings finished_sentence_embeddings feature_bert ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ) 0 78 List(List(document, 0, 77, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ), Map(sentence -> 0), List())) List(List(document, 0, 75, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor ., Map(sentence -> 0), List()), List(document, 77, 77, ), Map(sentence -> 1), List())) List(List(token, 0, 0, (, Map(sentence -> 0), List()), List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 42, 50-50, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List()), List(token, 75, 75, ., Map(sentence -> 0), List()), List(token, 77, 77, ), Map(sentence -> 0), List())) List(List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 41, 5050, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List())) List(List(word_embeddings, 2, 2, A, Map(isOOV -> false, pieceId -> 138, isWordStart -> true, token -> A, sentence -> 0), List(-0.0591245, 0.20256183, 0.8251248, 0.78046286, -1.7807933, -0.80155766, -1.1157473, -0.4782777, -0.11486364, -0.5550404, -0.4492663, -0.62475693, -0.9387575, -0.98547107, -0.0060980394, -1.7143714, 0.9728724, 0.8024508, -0.11146159, 0.27530736, -0.7498719, 0.3653813, -1.1369629, 0.7400096, 0.87870675, 0.67104024, -0.6720965, -0.97241056, -0.087701544, 0.11403816, -1.3198427, -2.1072302, -0.45286688, -0.95254016, 0.2114704, 0.32399213, -0.95956004, -0.67034465, -1.0145385, -0.1401028, 0.6138847, -0.008340925, -0.5912232, -0.042774387, -0.41416043, -0.305131, -0.5699813, 0.683128, 0.4442352, -1.2327226, -0.8284215, -0.6327381, -0.8909469, -0.34239066, -0.4762854, -1.5815666, 0.23999412, 0.6981139, -0.024432093, 0.14011878, -0.65106165, 0.7304999, 0.6718264, -0.27875805, 0.2163485, 0.33989796, -1.0445671, -0.06548089, 0.010210072, 0.247049, 0.6969329, -0.4022462, 0.77358055, -0.57319415, 0.42015353, 1.1350511, -0.16513473, -0.61211205, -0.4262871, -0.4906991, 0.73714745, -0.091579914, 1.0832553, 0.43265566, -0.9382083, 0.092016555, 0.12843782, -0.6982337, 0.17024484, 0.45182657, 0.30771467, 0.55378497, 0.34565946, -0.5830273, 1.3283966, 0.12674963, -0.23364174, 0.29585448, -0.5420519, 0.34525484, -0.5825339, -0.35446483, -0.2273812, -0.5216397, -1.2295258, 1.2310001, -0.43397206, -0.47614375, -0.8801553, -0.5638279, 0.63400173, -0.2587341, 0.10292122, -1.7655771, -0.2949517, 0.9170294, 0.54532516, 0.45913243, 0.32567474, -0.14687663, -0.5588464, -0.24423473, -1.1699482, 0.6929512, 0.66577256, 0.24080604, 0.41866362, 0.7635331, -0.3947026, 0.30512983, 0.8840208, 0.3868008, 0.6842996, 0.5227849, 0.22083575, 0.95

# 5 classifier and evaluation

In [52]:
from sklearn.metrics import classification_report, accuracy_score

In [53]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator # it's the primary machine learning API for spark for DataFrame-based API
from pyspark.mllib.evaluation import BinaryClassificationMetrics # after spark2.0 mllib enter maintenance mode

def print_performance_metrics(prediction):
  # evaluate model
  evaluator = BinaryClassificationEvaluator(rawPredictionCol='class')
  auc = evaluator.evaluate(prediction, {evaluator.metricName: "areaUnderROC"})
  aupr = evaluator.evaluate(prediction, {evaluator.metricName: "areaUnderPR"})
  print('auc = {}'.format(auc))
  print('aupr = {}'.format(aupr))

## 5.1 spark ML with elmo
* clf_lr_elmo_model

In [55]:
# spark ml classifier LogisticRegression with elmo
lr_elmo = LogisticRegression(labelCol='target', featuresCol='feature_elmo', maxIter=10)
lr_elmo_Model = lr_elmo.fit(train_elmo_embedding_df)

### EVALUATION: spark ML with elmo

In [57]:
lr_elmo_model_prediction = lr_elmo_Model.transform(test_elmo_embedding_df)

In [58]:
lr_elmo_model_prediction_df = lr_elmo_model_prediction.select('target', 'prediction').toPandas()
# lr_elmo_model_prediction_df['pred'] = lr_elmo_model_prediction_df['result'].apply(lambda x: x[0])
# lr_elmo_model_prediction_df['pred'] = lr_elmo_model_prediction_df['pred'].astype('int32')
print(classification_report(lr_elmo_model_prediction_df.target, lr_elmo_model_prediction_df.prediction))
print(accuracy_score(lr_elmo_model_prediction_df.target, lr_elmo_model_prediction_df.prediction))

precision recall f1-score support

 0 0.94 0.94 0.94 836
 1 0.86 0.86 0.86 376

 micro avg 0.91 0.91 0.91 1212
 macro avg 0.90 0.90 0.90 1212
weighted avg 0.91 0.91 0.91 1212

0.9133663366336634

## 5.2 spark ML with bert
* clf_lr_bert_model

In [60]:
# spark ml classifier LogisticRegression with bert
lr_bert = LogisticRegression(labelCol='target', featuresCol='feature_bert', maxIter=10)
lr_bert_Model = lr_bert.fit(train_bert_embedding_df)

### EVALUATION: spark ML with bert

In [62]:
lr_bert_model_prediction = lr_bert_Model.transform(test_bert_embedding_df)

In [63]:
lr_bert_model_prediction_df = lr_bert_model_prediction.select("target", "prediction").toPandas()
print(classification_report(lr_bert_model_prediction_df.target, lr_bert_model_prediction_df.prediction))
print(accuracy_score(lr_bert_model_prediction_df.target, lr_bert_model_prediction_df.prediction))

precision recall f1-score support

 0 0.94 0.95 0.94 836
 1 0.89 0.85 0.87 376

 micro avg 0.92 0.92 0.92 1212
 macro avg 0.91 0.90 0.91 1212
weighted avg 0.92 0.92 0.92 1212

0.9224422442244224

## 5.3 Spark-nlp DLApproach with ELMO embedding
* clf_elmo_model

In [65]:
# generate classifier by using elmo-embedding
clf_nlp_emlo_pipeline = Pipeline(stages=[document_assembler,
                                         sentence_detector,
                                         tokenizer,
                                         normalizer,
                                         word_embedding_elmo,
                                         sentence_embedding_elmo,
                                         classifierdl])

In [66]:
clf_elmo_model = clf_nlp_emlo_pipeline.fit(train_pb)

In [67]:
clf_elmo_prediction = clf_elmo_model.transform(test_pb)

### EVALUATION: spark-nlp-DLApproach ML with elmo

In [69]:
clf_elmo_prediction_df = clf_elmo_prediction.select('target', 'class.result').toPandas()

In [70]:
clf_elmo_prediction_df['pred'] = clf_elmo_prediction_df['result'].apply(lambda x: x[0])
clf_elmo_prediction_df['pred'] = clf_elmo_prediction_df['pred'].astype('int32')
print(classification_report(clf_elmo_prediction_df.target, clf_elmo_prediction_df.pred))
print(accuracy_score(clf_elmo_prediction_df.target, clf_elmo_prediction_df.pred))

precision recall f1-score support

 0 0.93 0.97 0.95 836
 1 0.92 0.84 0.88 376

 micro avg 0.93 0.93 0.93 1212
 macro avg 0.93 0.90 0.91 1212
weighted avg 0.93 0.93 0.93 1212

0.9273927392739274

## 5.4 Spark-nlp DLApproach with BERT embedding
* clf_bert_model

In [72]:
# generate classifier by using bert-embedding
clf_nlp_bert_pipeline = Pipeline(stages=[document_assembler,
                                         sentence_detector,
                                         tokenizer,
                                         normalizer,
                                         word_embedding_bert,
                                         sentence_embedding_bert,
                                         classifierdl])

In [73]:
# sparknlp DLApproach() with bert
clf_bert_model = clf_nlp_bert_pipeline.fit(train_pb)

In [74]:
clf_bert_prediction = clf_bert_model.transform(test_pb)

### EVALUATION: spark-nlp-DLApproach ML with bert

In [76]:
clf_bert_prediction_df = clf_bert_prediction.select('target', 'class.result').toPandas()

In [77]:
display(clf_bert_prediction)

text target pre_clean_len document sentence token normalized bert sentence_embeddings class ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ) 0 78 List(List(document, 0, 77, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor . ), Map(sentence -> 0), List())) List(List(document, 0, 75, ( A spokesperson told WWD to expect a 50-50 mix of clothing and home decor ., Map(sentence -> 0), List()), List(document, 77, 77, ), Map(sentence -> 1), List())) List(List(token, 0, 0, (, Map(sentence -> 0), List()), List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 42, 50-50, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List()), List(token, 75, 75, ., Map(sentence -> 0), List()), List(token, 77, 77, ), Map(sentence -> 0), List())) List(List(token, 2, 2, A, Map(sentence -> 0), List()), List(token, 4, 15, spokesperson, Map(sentence -> 0), List()), List(token, 17, 20, told, Map(sentence -> 0), List()), List(token, 22, 24, WWD, Map(sentence -> 0), List()), List(token, 26, 27, to, Map(sentence -> 0), List()), List(token, 29, 34, expect, Map(sentence -> 0), List()), List(token, 36, 36, a, Map(sentence -> 0), List()), List(token, 38, 41, 5050, Map(sentence -> 0), List()), List(token, 44, 46, mix, Map(sentence -> 0), List()), List(token, 48, 49, of, Map(sentence -> 0), List()), List(token, 51, 58, clothing, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 67, home, Map(sentence -> 0), List()), List(token, 69, 73, decor, Map(sentence -> 0), List())) List(List(word_embeddings, 2, 2, A, Map(isOOV -> false, pieceId -> 138, isWordStart -> true, token -> A, sentence -> 0), List(-0.0591245, 0.20256183, 0.8251248, 0.78046286, -1.7807933, -0.80155766, -1.1157473, -0.4782777, -0.11486364, -0.5550404, -0.4492663, -0.62475693, -0.9387575, -0.98547107, -0.0060980394, -1.7143714, 0.9728724, 0.8024508, -0.11146159, 0.27530736, -0.7498719, 0.3653813, -1.1369629, 0.7400096, 0.87870675, 0.67104024, -0.6720965, -0.97241056, -0.087701544, 0.11403816, -1.3198427, -2.1072302, -0.45286688, -0.95254016, 0.2114704, 0.32399213, -0.95956004, -0.67034465, -1.0145385, -0.1401028, 0.6138847, -0.008340925, -0.5912232, -0.042774387, -0.41416043, -0.305131, -0.5699813, 0.683128, 0.4442352, -1.2327226, -0.8284215, -0.6327381, -0.8909469, -0.34239066, -0.4762854, -1.5815666, 0.23999412, 0.6981139, -0.024432093, 0.14011878, -0.65106165, 0.7304999, 0.6718264, -0.27875805, 0.2163485, 0.33989796, -1.0445671, -0.06548089, 0.010210072, 0.247049, 0.6969329, -0.4022462, 0.77358055, -0.57319415, 0.42015353, 1.1350511, -0.16513473, -0.61211205, -0.4262871, -0.4906991, 0.73714745, -0.091579914, 1.0832553, 0.43265566, -0.9382083, 0.092016555, 0.12843782, -0.6982337, 0.17024484, 0.45182657, 0.30771467, 0.55378497, 0.34565946, -0.5830273, 1.3283966, 0.12674963, -0.23364174, 0.29585448, -0.5420519, 0.34525484, -0.5825339, -0.35446483, -0.2273812, -0.5216397, -1.2295258, 1.2310001, -0.43397206, -0.47614375, -0.8801553, -0.5638279, 0.63400173, -0.2587341, 0.10292122, -1.7655771, -0.2949517, 0.9170294, 0.54532516, 0.45913243, 0.32567474, -0.14687663, -0.5588464, -0.24423473, -1.1699482, 0.6929512, 0.66577256, 0.24080604, 0.41866362, 0.7635331, -0.3947026, 0.30512983, 0.8840208, 0.3868008, 0.6842996, 0.5227849, 0.22083575, 0.95339185, -1.3704709, 0.50894976, -0.7

In [78]:
clf_bert_prediction_df['pred'] = clf_bert_prediction_df['result'].apply(lambda x: x[0])
clf_bert_prediction_df['pred'] = clf_bert_prediction_df['pred'].astype('int32')
print(classification_report(clf_bert_prediction_df.target, clf_bert_prediction_df.pred))
print(accuracy_score(clf_bert_prediction_df.target, clf_bert_prediction_df.pred))

/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
 precision recall f1-score support

 0 0.69 1.00 0.82 836
 1 0.00 0.00 0.00 376

 micro avg 0.69 0.69 0.69 1212
 macro avg 0.34 0.50 0.41 1212
weighted avg 0.48 0.69 0.56 1212

0.6897689768976898